In [133]:
# https://github.com/RaRe-Technologies/gensim/blob/develop/docs/notebooks/doc2vec-lee.ipynb
from collections import namedtuple

from gensim.summarization.textcleaner import get_sentences
from gensim.models.doc2vec import TaggedDocument, Doc2Vec


class TaggedDocument(namedtuple('TaggedDocument', 'words tags original')):
    """ Based on gensim's TaggedDocument. "original" field is added. """
    # FIXME: Extend class to keep all methods.
    
    def __str__(self):
        return '%s(%s, %s)' % (self.__class__.__name__, self.words, self.tags)


def read_corpus(fname):
    with open(fname) as f:
        text = f.read()
        text = text.replace('\n', '')
        for i, sentence in enumerate(get_sentences(text)):
            yield TaggedDocument(gensim.utils.simple_preprocess(sentence), [i], sentence)

In [134]:
train_corpus = list(read_corpus('book.txt'))

In [135]:
model = Doc2Vec(vector_size=50, min_count=2, epochs=40, train_lbls=False)
model.build_vocab(train_corpus)
model.train(train_corpus, total_examples=model.corpus_count, epochs=model.epochs)

In [136]:
target_id = 0
print('Target: {}'.format(train_corpus[target_id].original))

similarity = {}
for i, sentence in enumerate(train_corpus):
    similarity[i] = model.docvecs.similarity(target_id, i)
    
sorted_similarity = sorted(similarity.items(), key=lambda item: item[1], reverse=True)

for i, value in sorted_similarity[:10]:
    print('[{}] {}'.format(value, train_corpus[i].original))

Target: A rough sketch of the circumstances which led up to my being taken aprisoner of war are more or less indispensable.
[1.0] A rough sketch of the circumstances which led up to my being taken aprisoner of war are more or less indispensable.
[0.9776094555854797] The more escapes, the more guards necessary to prevent them, themore electric lights or oil lamps to show up the designs of the escapersby night, the continual supply of coal and oil necessary to feed theselights, slowly but very surely help to drain the resources of theBoches.
[0.9711006283760071] Theywere certainly boundary-posts of some kind.
[0.9675556421279907] Do not unlink or detach or remove the full Project Gutenberg-tmLicense terms from this work, or any files containing a part of thiswork or any other work associated with Project Gutenberg-tm.1.E.5.
[0.961080014705658] YOU AGREE THAT YOU HAVE NO REMEDIES FOR NEGLIGENCE, STRICTLIABILITY, BREACH OF WARRANTY OR BREACH OF CONTRACT EXCEPT THOSEPROVIDED IN PARAGRAPH 1.

/Users/admin/Workspace/test_project/venv/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):
